In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
import json
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from tqdm import tqdm
import pymysql
from sqlalchemy import create_engine

# dask packages

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client

In [2]:
# dir 설정

filename = 'payload_playerherostats-20200831-20200906.tsv/payload_playerherostats-20200831-20200906.tsv'

In [3]:
df = pd.read_csv('to_alien.csv')
df

# ddf = pd.read_csv('payload_gameinfo-20200831-20200906.tsv.gz', nrows = 3)
# ddf

,Unnamed: 0,time_c,schema_name,time,info,player_id,hero_guid,stat_lifespan,stat,team_id,context,player,team,esports_match_id
0,0,2020-09-03 19:09:16,PlayerHeroStats,1599160156569,"{""event_id"":51,""sequence_length"":116,""sequence...","{""src"":110,""seq"":8028075772644456808}",207165582859042824,ROUND,"{""short_stat_guid"":1472,""amount"":462.07001128125}",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":110,""seq"":8028075772644456...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
1,1,2020-09-03 19:09:18,PlayerHeroStats,1599160158654,"{""event_id"":56,""sequence_length"":116,""sequence...","{""src"":0,""seq"":7452947632662870381}",207165587154010111,MAP_OBJECTIVE,"{""short_stat_guid"":1136,""amount"":1.0}",d770916a-d2f9-2f49-9bcb-d5585698fdc2,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":745294763266287038...","{""team_id"":""d770916a-d2f9-2f49-9bcb-d5585698fd...",35535
2,2,2020-09-03 19:09:20,PlayerHeroStats,1599160160733,"{""event_id"":61,""sequence_length"":116,""sequence...","{""src"":0,""seq"":114784954376548}",207165582859042819,ROUND,"{""short_stat_guid"":1497,""amount"":0.02777777798...",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":114784954376548},""...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
3,3,2020-09-03 19:09:22,PlayerHeroStats,1599160162815,"{""event_id"":67,""sequence_length"":116,""sequence...","{""src"":0,""seq"":435727461474}",972777519512029264,MAP_OBJECTIVE,"{""short_stat_guid"":1812,""amount"":12.6736896634...",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":435727461474},""bat...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
4,4,2020-09-03 19:09:22,PlayerHeroStats,1599160162815,"{""event_id"":67,""sequence_length"":116,""sequence...","{""src"":120,""seq"":7307216244114747235}",207165582859042819,ROUND,"{""short_stat_guid"":1622,""amount"":0.0}",d770916a-d2f9-2f49-9bcb-d5585698fdc2,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":120,""seq"":7307216244114747...","{""team_id"":""d770916a-d2f9-2f49-9bcb-d5585698fd...",35535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2020-09-03 19:20:52,PlayerHeroStats,1599160852272,"{""event_id"":1350,""sequence_length"":65,""sequenc...","{""src"":110,""seq"":8028075772644456808}",207165582859042824,GAME,"{""short_stat_guid"":1235,""amount"":0.02113633729...",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":110,""seq"":8028075772644456...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
996,996,2020-09-03 19:20:52,PlayerHeroStats,1599160852272,"{""event_id"":1350,""sequence_length"":65,""sequenc...","{""src"":0,""seq"":435727590762}",207165587154010111,GAME,"{""short_stat_guid"":668,""amount"":0.123775569045...",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":435727590762},""bat...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
997,997,2020-09-03 19:20:56,PlayerHeroStats,1599160856434,"{""event_id"":1356,""sequence_length"":65,""sequenc...","{""src"":0,""seq"":435727461474}",207165582859042880,GAME,"{""short_stat_guid"":1214,""amount"":236.559434592...",34c3e757-3ed1-b747-afc4-c1cef7266ed1,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":435727461474},""bat...","{""team_id"":""34c3e757-3ed1-b747-afc4-c1cef7266e...",35535
998,998,2020-09-03 19:20:56,PlayerHeroStats,1599160856434,"{""event_id"":1356,""sequence_length"":65,""sequenc...","{""src"":0,""seq"":32491047111325030}",207165582859042880,GAME,"{""short_stat_guid"":796,""amount"":2.0}",d770916a-d2f9-2f49-9bcb-d5585698fdc2,"{""type"":""SNAPSHOT""}","{""player_id"":{""src"":0,""seq"":32491047111325030}...","{""team_id"":""d770916a-d2f9-2f49-9bcb-d5585698fd...",35535


In [4]:
pbar = ProgressBar()
pbar.register()

num_cores = mp.cpu_count()

ddf = dd.read_csv(filename, delimiter = '\t', compression = 'gzip')
ddf = ddf[ddf['esports_match_id'] == 35535]
ddf.head()
# ddf['time'].count().compute(scheduler = 'processes', num_workers = num_cores)

[                                        ] | 0% Completed | 22.7s
[                                        ] | 0% Completed | 22.7s


KeyboardInterrupt: 

In [3]:
# interest headers 지정

headers = ['time', 'hero_guid', 'stat_lifespan', 'stat', 'player', 'team', 'esports_match_id']
json_type_headers = ['stat', 'player', 'team']
stat_json_interest_headers = ['short_stat_guid', 'amount']
player_json_interest_headers = ['battletag', 'esports_player_id']
team_json_interest_headers = ['esports_team_id']


def append_json_to_df_phs(df_phs, json_type_headers):
    
    tmp_stat = pd.DataFrame(columns = stat_json_interest_headers)
    tmp_player = pd.DataFrame(columns = player_json_interest_headers)
    tmp_team = pd.DataFrame(columns = team_json_interest_headers)
    for header in json_type_headers:
        for idx in df_phs.index.values:
            if header == 'stat':
                json_text = json.loads(df_phs.loc[idx, header])
                json_norm = pd.json_normalize(json_text)
                tmp_stat = pd.concat([tmp_stat, json_norm], ignore_index = True)
            elif header == 'player':
                json_text = json.loads(df_phs.loc[idx, header])
                json_norm = pd.json_normalize(json_text)[player_json_interest_headers]
                tmp_player = pd.concat([tmp_player, json_norm], ignore_index = True)
            elif header == 'team':
                json_text = json.loads(df_phs.loc[idx, header])
                json_norm = pd.json_normalize(json_text)[team_json_interest_headers]
                tmp_team = pd.concat([tmp_team, json_norm], ignore_index = True)
    
    df_phs = pd.concat([df_phs, tmp_stat, tmp_player, tmp_team], axis = 1)
    df_phs.drop(columns = json_type_headers, inplace = True) 

    return df_phs


In [ ]:
# chunksize = 10000

# df_chunk = pd.read_csv(filename, delimiter = '\t', chunksize = chunksize)

# start_time = time.time()

# for chunk in tqdm(df_chunk, desc = 'num chunk saved'):
#     pass
    
#     # df_sample = chunk[headers]
#     # df_sample.reset_index(inplace = True)
#     # print(df_sample.size)
#     # new_df = append_json_to_df_phs(df_sample, json_type_headers)
    
#     # if new_df.isnull().values.any() == False:
#     #     pass
#     # else:
#     #     print('table에 nan값이 포함되어있습니다')
    
#     # # Credentials to DB connection

#     # hostname = "localhost" 
#     # username = "root"
#     # pwd = "gpdlzjadh"
#     # dbname = "esd_phs"
#     # charset = "utf8"

#     # # Create SQLAlchemy engine to connect to MySQL DB

#     # engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host = hostname, db = dbname, user = username, pw = pwd))

#     # # Convert df to sql table

#     # new_df.to_sql(name = dbname, con = engine, if_exists = 'append', index = False, method = 'multi')

# end_time = time.time()- start_time
# print(end_time)